In [309]:
import pandas
import random
import os
import json
import numpy as np

ROWCOUNT = 1000

## Part 1

### Customer

In [310]:
custFileName = "nameList1000.txt"

ROOT = os.path.dirname(os.getcwd()) + '/resource/' + custFileName

first_names = []
last_names = []

with open(ROOT, 'r') as file:
    rawnames = file.read().split('\n')

for name in rawnames:
    first_names.append(name[:name.find(' ')])
    last_names.append(name[1+name.find(' '):])

In [311]:
'''
--
-- given list of 1st and 2nd names
-- randomize them and duplicate them randomly for repeat customers
--
'''

# Create a list to hold all names
names = []

# Generate unique names (50% of 4000 = 2000 unique names)
for index in range(2000):
    first = random.choice(first_names)
    last = random.choice(last_names)
    name = f"{first} {last}"
    names.append(name)

# Generate repeated names (remaining 50% will be repeats 5-10 times)
for index in range(ROWCOUNT):  # Need 1000 more distinct names, each repeating 5-10 times
    first = random.choice(first_names)
    last = random.choice(last_names)
    name = f"{first} {last}"
    repeat_count = random.randint(5, 10)  # Repeat each name 5 to 10 times
    names.extend([name] * repeat_count)

# Shuffle the final list to mix unique and repeated names
random.shuffle(names)

# Ensure the list is exactly 1000 names
names = names[:ROWCOUNT]

# Convert the list to a single line with commas separating each name
# result = ", ".join(names)

In [312]:
'''
--
-- DF to pandas
--
'''
customerDict = {}
first_names = []
last_names = []

for name in names:
    first_names.append(name[:name.find(' ')])
    last_names.append(name[1+name.find(' '):])

customerDict["First"] = first_names
customerDict["Last"] = last_names

customerDF = pandas.DataFrame.from_dict(data=customerDict)
customerDF


,First,Last
0,Emily,Myers
1,Olivia,Bennett
2,Maya,Lewis
3,Ella,Hernandez
4,Maya,Perez
...,...,...
995,Caroline,Baker
996,Iris,Rodriguez
997,Paisley,Castillo
998,Delilah,Ramirez


### Store

In [313]:
generateData = "genData.json"

ROOT = os.path.dirname(os.getcwd()) + '/resource/' + generateData

storeDict = {}

with open(ROOT, 'r') as file:
    data = json.load(file)
    data = data["Store"]

listAdd = []
for dict in data:
    listAdd.append(data[dict])

    
storeDf = pandas.DataFrame.from_records(listAdd)
storeDf

,StoreId,Name,Address,City,Texas,Zip Code
0,1,South Austin Drive,102 South Austin Drive,Austin,Texas,72034
1,2,Western Drive,111 Western Drive,Houston,Texas,99345
2,3,North Lake,880 North Lake Drive,Dallas,Texas,14783


### Employee

In [314]:
generateData = "genData.json"

ROOT = os.path.dirname(os.getcwd()) + '/resource/' + generateData

with open(ROOT, 'r') as file:
    data = json.load(file)
    data = data["Employee"]

# get each stores employees
stores = []
for store in data:
    stores.append(data[store])


In [315]:
'''
{'one': {'EmployeeID': '1', 'ManagerID': 'Null', 'FirstName': 'Tom', 'LastName': 'Bates', 'StoreID': '1'}, 'two': {
'''


df = pandas.DataFrame(stores[0]).T
stores.pop(0)

for data in stores:
    dfNew = pandas.DataFrame(data).T
    df = pandas.concat([df,dfNew])

Employeedf = df
Employeedf.reset_index(drop=True, inplace=True)


### Product

In [316]:
generateData = "genData.json"

ROOT = os.path.dirname(os.getcwd()) + '/resource/' + generateData

with open(ROOT, 'r') as file:
    data = json.load(file)
    data = data["Product"]

# get each stores employees
product = []
for products in data:
    product.append(data[products])

productDF = pandas.DataFrame.from_records(product).T

productDF = productDF.rename(columns={0: 'Product', 1: 'Price'})

productDF['productID'] = productDF.reset_index().index + 1


### Full Table DF - Store

In [317]:
# Generate random integers between 0 and 10 for each row
random_counts = np.random.randint(0, 21, size=len(storeDf))
# random_counts

while storeDf.shape[0] < ROWCOUNT:
    
    # for each row number, count # of rows to add to df
    for row, count in enumerate(random_counts):
        # storeDf.concat([storeDf.loc[[i]] * count])
        row_to_duplicate = storeDf.loc[[row]]

        # Duplicate the row 2 times
        duplicated_row = pandas.concat([row_to_duplicate] * count, ignore_index=True)

        # Create a new DataFrame with the duplicated rows
        storeDf = pandas.concat([storeDf, duplicated_row], ignore_index=True)

# limit to 1000 rows
storeDf = storeDf.iloc[:ROWCOUNT]

### Full Table DF - Employee

In [318]:
# Generate random integers between 0 and 10 for each row
random_counts = np.random.randint(0, 21, size=len(Employeedf))
random_counts


array([ 3,  6,  4, 19, 18,  7, 17,  3, 10,  7, 19,  4,  2,  2])

In [319]:

while Employeedf.shape[0] < ROWCOUNT:
    
    # for each row number, count # of rows to add to df
    for row, count in enumerate(random_counts):
        # storeDf.concat([storeDf.loc[[i]] * count])
        row_to_duplicate = Employeedf.loc[[row]]

        # Duplicate the row 2 times
        duplicated_row = pandas.concat([row_to_duplicate] * count, ignore_index=True)

        # Create a new DataFrame with the duplicated rows
        Employeedf = pandas.concat([Employeedf, duplicated_row], ignore_index=True)

# limit to 1000 rows
Employeedf = Employeedf.iloc[:ROWCOUNT]

# Full table Df - Product

In [320]:
# Generate random integers between 0 and 10 for each row
random_counts = np.random.randint(0, 21, size=len(productDF))
random_counts


while productDF.shape[0] < ROWCOUNT:
    
    # for each row number, count # of rows to add to df
    for row, count in enumerate(random_counts):
        # storeDf.concat([storeDf.loc[[i]] * count])
        row_to_duplicate = productDF.loc[[row]]

        # Duplicate the row 2 times
        duplicated_row = pandas.concat([row_to_duplicate] * count, ignore_index=True)

        # Create a new DataFrame with the duplicated rows
        productDF = pandas.concat([productDF, duplicated_row], ignore_index=True)

# limit to 1000 rows
productDF = productDF.iloc[:ROWCOUNT]

## Concat ALl Tables into 1 table

In [321]:
fullTableDF = pandas.concat([customerDF,storeDf], axis=1)
fullTableDF = pandas.concat([fullTableDF,Employeedf], axis=1)
fullTableDF = pandas.concat([fullTableDF,productDF], axis=1)

In [322]:
fullTableDF

,First,Last,StoreId,Name,Address,City,Texas,Zip Code,EmployeeID,ManagerID,FirstName,LastName,StoreID,Product,Price,productID
0,Emily,Myers,1,South Austin Drive,102 South Austin Drive,Austin,Texas,72034,1,Null,Tom,Bates,1,Hot Cocoa,4.5,1
1,Olivia,Bennett,2,Western Drive,111 Western Drive,Houston,Texas,99345,2,1,Alex,Dune,1,Espresso,3,2
2,Maya,Lewis,3,North Lake,880 North Lake Drive,Dallas,Texas,14783,3,1,Frank,Watts,1,Ristretto,3.5,3
3,Ella,Hernandez,1,South Austin Drive,102 South Austin Drive,Austin,Texas,72034,4,1,Daniel,Farmer,1,Americano,4,4
4,Maya,Perez,1,South Austin Drive,102 South Austin Drive,Austin,Texas,72034,5,Null,Toby,Harts,2,Latte,5.5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Caroline,Baker,1,South Austin Drive,102 South Austin Drive,Austin,Texas,72034,4,1,Daniel,Farmer,1,Nitro Brew,6.5,14
996,Iris,Rodriguez,1,South Austin Drive,102 South Austin Drive,Austin,Texas,72034,4,1,Daniel,Farmer,1,Nitro Brew,6.5,14
997,Paisley,Castillo,1,South Austin Drive,102 South Austin Drive,Austin,Texas,72034,4,1,Daniel,Farmer,1,Nitro Brew,6.5,14
998,Delilah,Ramirez,1,South Austin Drive,102 South Austin Drive,Austin,Texas,72034,4,1,Daniel,Farmer,1,Nitro Brew,6.5,14


## GCP Push